# Usage in Python
- Python has support for working with different databases:
1. sqlite
2. mysql
3. postgres
4. etc.

## Psycopg
- Psycopg is the most popular PostgreSQL database adapter for python
- Psycopg 2 is mostly implemented in C
- Many Python types are supported and adapted to matching PostgreSQL data types

### Basic module usage

> python3 -m pip install psycopg2

> create database python_db;

In [27]:
import psycopg2

#Connect to an existing database
conn = psycopg2.connect(
    host="localhost", #or
    #host="127.0.0.1", 
    database="python_db",
    user='postgres',
    password='password'
)

#Open a cursor to perform database operations
cursor = conn.cursor()

#Execute a command: this creates a new table

# try:
#     cursor.execute("create table test (id serial primary key, num integer, data varchar(50))")
# except Exception as e:
#     print('Error:', e)

cursor.execute("create table if not exists test (id serial primary key, num integer, data varchar(50))")

#Pass data to fill a query placeholders 
cursor.execute("insert into test (num, data) values (%s, %s)", (100, "abc'def"))

#Query the database and obtain data as python objects 
cursor.execute("select * from test;")
data = cursor.fetchall()

# Make the change to the database persistent
conn.commit()

#Close communication wih the database
cursor.close()
#conn.close()
data



[(1, 100, "abc'def"),
 (2, 100, "abc'def"),
 (4, 100, "abc'def"),
 (5, 100, "abc'def"),
 (6, 100, "abc'def"),
 (7, 100, "abc'def"),
 (8, 100, "abc'def"),
 (9, 100, "abc'def"),
 (10, 100, "abc'def")]

- connect() creates a new database session

In [25]:
print(type(conn))

<class 'psycopg2.extensions.connection'>


- each connection/session can create new cursor instances
- cursor instances can execute database commands
- commands are sent by the following methods:
1. execute()
2. executemany()
- to retrieve data use the following cursor methods:
1. fetchall() - gets all row of the query
2. fetchone() - gets one row of the query
3. fetchmany() - gets as many rows as specified e.g. fetchmany(5)

### Access Data From a cursor
- You can have many cursors sharing the same connection to a database.
- Cursors created from the same connection are not isolated
- i.e. any changes done to the database by a cursor are visible by other cursor

- a query can potentially match very large sets of data
- Read operations do not immediately return all values matching the query
- these operations rely on cursors

- cursors fetches data in batches to reduce memory consumption

- Closing the cursor frees resources associated to the queries.
- It would not eliminate the connection to the database itself.
(Therefore, there is no need for reauthentication)



### Connections and Cursors
- Connections and cursors can be used as context managers and commits if no exception occurs. 
- the connection is not closed by the context
- the cursor is closed by the context

In [32]:
with conn:
    with conn.cursor() as curs:
        curs.execute("CREATE TABLE if not exists test3 (id serial PRIMARY KEY, num integer, data varchar)")
        curs.execute('select * from test')
        data = curs.fetchall()
        print(curs)
print(curs)
#conn.commit happens
#curs.close() happens
data


<cursor object at 0x7fca4f39eb80; closed: 0>
<cursor object at 0x7fca4f39eb80; closed: -1>


[(1, 100, "abc'def"),
 (2, 100, "abc'def"),
 (4, 100, "abc'def"),
 (5, 100, "abc'def"),
 (6, 100, "abc'def"),
 (7, 100, "abc'def"),
 (8, 100, "abc'def"),
 (9, 100, "abc'def"),
 (10, 100, "abc'def")]

### Passing parameters to SQL queries
### execute and executemany()
- Execute a database operation 
- Parameters may be provided 
- For positional variables binding, the second argument must always be a sequence

In [33]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%s, %s)
        """, (22, 'BLA'))
        curs.execute("select * from test")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA')]


- Named arguments are supported too using %(name)s placeholders in the query
- allows to specify the values in any order and to repeat the same value

In [34]:
with conn:
    with conn.cursor() as curs:
        curs.execute("""
        insert into test (num, data)
        values (%(value1)s, %(value2)s)
        """, {'value1':100, 'value2':'DATA'})
        curs.execute("select * from test")
        print(curs.fetchall())

[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA')]


### The problem with the query parameters - SQL Injection

In [47]:
with conn:
   with conn.cursor() as curs:
      SQL = "insert into test (data) values ('%s');" # don't use quotes around %s
      #user_input = "Bla'); select * from test where '1' in ('1"
      user_input = input('please give data to store')
      curs.execute(SQL % user_input) #NEVER DO THIS
      print(curs.fetchall())


[(1, 100, "abc'def"), (2, 100, "abc'def"), (4, 100, "abc'def"), (5, 100, "abc'def"), (6, 100, "abc'def"), (7, 100, "abc'def"), (8, 100, "abc'def"), (9, 100, "abc'def"), (10, 100, "abc'def"), (11, 22, 'BLA'), (12, 100, 'DATA'), (16, 100, "abc'def"), (17, 100, 'DATA'), (19, None, 'Bla'), (20, None, 'Bla'), (21, None, 'Bla'), (22, None, 'Bla'), (25, None, 'Bla')]


In [35]:
'Hello %s' % 'bla'

'Hello bla'

In [58]:
with conn:
   with conn.cursor() as curs:
      SQL = "insert into test (data) values (%s);" # don't use quotes around %s
      user_input = "HELLO WORLD'); select * from test where '1' in ('1"
      #user_input = 'HELLO WORLD'
      #user_input = input('please give data to store')
      curs.execute(SQL, (user_input,)) # DO IT THIS WAY
      

<span style=color:red;>Warning: Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. </span>

In [ ]:
SQL = "insert into test (data) values (%s);"

![](exploits_of_a_mom.png)

### executemany



In [61]:
with conn:
    with conn.cursor() as curs:
        number = (1000,)
        numbers = [(1000,), (2000,), (3000,)]
        SQL = "insert into test (num) values (%s)"
        # curs.execute(SQL, number) #single insert
        curs.executemany(SQL, numbers) # multiple inserts 



- Parameters are bounded to the query using the same rules described in the execute() method.
- In its current implementation this method is not faster than executing execute() in a loop